In [30]:
import pandas as pd
import tensorflow as tf
from datasets import load_dataset
from sklearn.model_selection import train_test_split

#most code is taken from the old huggingface script for language modeling with 


In [31]:
# create functions to read in JSON data of transcripts
datasets = load_dataset("kpriyanshu256/whisper-transcripts")




Using custom data configuration kpriyanshu256--whisper-transcripts-b6043c49b20f7241
Found cached dataset json (/Users/ArjunPatel/.cache/huggingface/datasets/kpriyanshu256___json/kpriyanshu256--whisper-transcripts-b6043c49b20f7241/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 1/1 [00:00<00:00, 396.66it/s]


In [34]:
datasets["train"]

Dataset({
    features: ['id', 'text', 'segments'],
    num_rows: 799
})

In [32]:
text_data = datasets["train"]["text"]

In [35]:
import transformers
model_checkpoint = "distilgpt2"
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenize_function(examples):
    return tokenizer(examples["text"])


tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns = ["text", "id", "segments"])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]


#1: 100%|██████████| 1/1 [00:00<00:00, 13.67ba/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (1255 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1886 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2072 > 1024). Running this sequence through the model will result in indexing errors
#3: 100%|██████████| 1/1 [00:00<00:00, 12.56ba/s]


In [36]:
# tokenize and preprocess dataset

def group_texts(examples):
    block_size = 64
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [37]:
# set up distilgpt2 model

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0: 100%|██████████| 1/1 [00:00<00:00, 20.63ba/s]

#1: 100%|██████████| 1/1 [00:00<00:00, 23.01ba/s]


#3: 100%|██████████| 1/1 [00:00<00:00, 25.59ba/s]


In [38]:
from transformers import TFAutoModelForCausalLM

model = TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(lr=2e-5, weight_decay_rate=0.01)
import tensorflow as tf

model.compile(optimizer=optimizer)

2022-10-16 18:15:57.861231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
/Users/ArjunPatel/opt/anaconda3/envs/whisper-gpt/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
No loss specified in compile() - the model's internal loss computation will be used as the loss. 

In [39]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [40]:
mod_history = model.fit(train_set, epochs=1)

41/41 [==============================] - 113s 2s/step - loss: 3.8653


TypeError: 'History' object is not subscriptable